In [2]:
from xgboost import XGBClassifier
from matplotlib import pyplot
import pandas as pd
import numpy as np

from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer

In [67]:
df = pd.read_csv('/home/hawliet/Documents/Data-science-project/data/cars/cars_preprocessed_undrop.csv', sep='\t', encoding='utf-8')
df.head(3)

,url,name,model,brand,price,eLabel,bodyType,length,height,width,...,fuelCapacity,fuelConsumption,speed,payload,trailerWeight,vEengineType,vEfuelType,vEengineDisplacement,vEenginePower,torque
0,http://www.cars-data.com//en/audi-s3-quattro-s...,Audi S3 quattro,Audi S3,Audi,42.247,G,hatchback,4159.0,1415.0,1763.0,...,55.0,9.1,238.0,560.0,1600.0,dohc,petrol,1781.0,154.0,270.0
1,http://www.cars-data.com//en/audi-a6-avant-40-...,Audi A6 Avant 40 TDI,Audi A6 Avant,Audi,64.450,C,stationwagon,4939.0,1494.0,1886.0,...,73.0,4.5,241.0,645.0,2000.0,dohc,diesel,1968.0,0.0,400.0
2,http://www.cars-data.com//en/audi-a6-avant-45-...,Audi A6 Avant 45 TDI quattro,Audi A6 Avant,Audi,77.130,E,stationwagon,4939.0,1494.0,1886.0,...,73.0,5.7,250.0,695.0,2000.0,dohc,diesel,2967.0,0.0,500.0


In [68]:
df.drop(columns=['url', 'name', 'model', 'fuelType', 'vehicleTransmission'], inplace=True)

In [69]:
num_cols = ['length', 'height', 'width', 'weight', 'weightTotal', 'emissionsCO2', 'numberOfAxles',
            'numberOfDoors', 'numberOfForwardGears', 'seatingCapacity', 'cargoVolume', 'roofLoad', 
            'accelerationTime', 'fuelCapacity', 'fuelConsumption', 'speed', 'payload', 'trailerWeight', 
            'vEengineDisplacement', 'vEenginePower', 'torque']
cat_cols = []
for col in df.columns:
    if not (col in num_cols):
        cat_cols.append(col)

In [128]:
df[cat_cols][11100:11125]

,brand,price,eLabel,bodyType,modelDate,driveWheelConfiguration,vEengineType,vEfuelType
11100,Mitsubishi,14.990,B,hatchback,2013.0,front,dohc,petrol
11101,Mitsubishi,13.590,B,hatchback,2013.0,front,dohc,petrol
11102,Mitsubishi,13.990,B,hatchback,2013.0,front,dohc,petrol
11103,Mitsubishi,12.490,B,hatchback,2013.0,front,dohc,petrol
11104,Mitsubishi,10.490,B,hatchback,2013.0,front,dohc,petrol
11105,Mitsubishi,8.990,B,hatchback,2013.0,front,dohc,petrol
11106,Mitsubishi,13.990,B,hatchback,2013.0,front,dohc,petrol
11107,Mitsubishi,12.490,B,hatchback,2013.0,front,dohc,petrol
11108,Mitsubishi,10.490,B,hatchback,2013.0,front,dohc,petrol
11109,Mitsubishi,8.990,B,hatchback,2013.0,front,dohc,petrol


In [184]:
df.vEfuelType.unique()

array(['petrol', 'diesel', 'LPG', 'benzine', 'bio-ethanol', 'natural gas',
       nan, 'LPG / petrol', 'aardgas', 'aardgas / petrol',
       'petrol / bio-ethanol'], dtype=object)

In [205]:
tmp = df.driveWheelConfiguration.str.replace('[^\w]', ' ').str.split()
types = set()

for i in range(tmp.shape[0]):
    if type(tmp[i]) is list:
        types.add(tmp[i][0])

In [206]:
a = list(types)
a

['achter', 'rear', 'voor', 'front']

In [166]:
tmp[0]

['petrol']

In [159]:
type

False

In [147]:
val = tmp[1]
if type(val) is list:
    for i in range(len(val)):
        if a[0] in val:
            print('1')
            break
else:
    print('0')

In [165]:
def check_label_existed(val):
    if (type(val) is list) and ('LPG' in val):
        print('1')
    else:
        print('0')

1


In [209]:
a

['achter', 'rear', 'voor', 'front']

In [224]:
tmp_df = df[['vEfuelType', 'driveWheelConfiguration']].copy()
tmp_df.head(1)

,vEfuelType,driveWheelConfiguration
0,petrol,front+rear


In [220]:
Counter(df.driveWheelConfiguration.apply(lambda x: 1 if ((type(x) is str) and (a[3] in x)) else 0))

Counter({1: 70658, 0: 13516})

In [221]:
for i in range(len(a)):
    tmp_df['driveWheelConfig' + '_' + a[i]] = tmp_df.driveWheelConfiguration.apply(lambda x: 1 if ((type(x) is str) and (a[i] in x)) else 0)

In [233]:
type(tmp_df.vEfuelType[0])

str

In [239]:
class MultilabelEcoding(BaseEstimator, TransformerMixin):
    def __init__(self, col_name):
        self.col_name = col_name
        self.type_list = set()
    def fit(self, X_df, y=None):
        types_sr = X_df[self.col_name].str.replace('[^\w]', ' ').str.split()
        for i in range(types_sr.shape[0]):
            if type(types_sr[i]) is list:
                self.type_list.add(types_sr[i][0])
        self.type_list = list(self.type_list)
        return self
    def transform(self, X_df, y=None):
        transformed_df = X_df.copy()
        for i in range(len(self.type_list)):
            transformed_df[self.col_name + '_' + self.type_list[i]] = transformed_df[self.col_name].apply(lambda x: 1 if ((type(x) is str) and (self.type_list[i] in x)) else 0)
        return transformed_df

In [242]:
multi_encoder = MultilabelEcoding('driveWheelConfiguration')
multi_encoder.fit(tmp_df)
multi_encoder.type_list

['achter', 'rear', 'voor', 'front']

In [243]:
multi_encoder.transform(tmp_df)

,vEfuelType,driveWheelConfiguration,driveWheelConfiguration_achter,driveWheelConfiguration_rear,driveWheelConfiguration_voor,driveWheelConfiguration_front
0,petrol,front+rear,0,1,0,1
1,diesel,front,0,0,0,1
2,diesel,front+rear,0,1,0,1
3,diesel,front+rear,0,1,0,1
4,petrol,front,0,0,0,1
...,...,...,...,...,...,...
84169,petrol,front+rear,0,1,0,1
84170,petrol,front+rear,0,1,0,1
84171,petrol,front+rear,0,1,0,1
84172,petrol,front+rear,0,1,0,1


In [16]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

column_trans = ColumnTransformer([('cats', categorical_transformer, cat_cols)])
# ('nums', SimpleImputer(missing_values=np.nan, strategy='mean'), num_cols)

preprocess_pipeline = Pipeline(steps=[
    ('colsTransformer', column_trans)])
# ('stdScaler', StandardScaler())

X = preprocess_pipeline.fit_transform(df[cat_cols])

In [17]:
si = SimpleImputer(strategy='mean')
y = si.fit_transform(df.price.values.reshape(-1, 1))

In [18]:
X.shape, y.shape

((84174, 27322), (84174, 1))

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9)

In [20]:
X_train.shape

(8417, 27322)

In [ ]:
model = XGBClassifier()
model.fit(X, y)

/home/hawliet/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/hawliet/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
